<a href="https://colab.research.google.com/github/anjalinagel12/NLP/blob/master/Zero_Shot_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-skm3jrli
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-skm3jrli
     |████████████████████████████████| 3.0MB 6.5MB/s 
     |████████████████████████████████| 1.1MB 41.6MB/s 
     |████████████████████████████████| 890kB 36.9MB/s 
  Created wheel for transformers: filename=transformers-3.0.2-cp36-none-any.whl size=867670 sha256=de6263c4cedcf16eb1f68afc5a129ac2f9e5d75446d0d8552964c113571e7fd6
  Stored in directory: /tmp/pip-ephem-wheel-cache-sd_zjtjx/wheels/33/eb/3b/4bf5dd835e865e472d4fc0754f35ac0edb08fe852e8f21655f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=737e43648d0d1a7852c670f0e2cc413dea96e58e80a4eb2e99a4ba9e9a6a2f78
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


In [2]:
from transformers import pipeline

In [3]:
classifier = pipeline("zero-shot-classification")

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.

## Rasa Analysis

In [4]:
text=input("PLEASE ENTER your sentence here: ")

PLEASE ENTER your sentence here: ill make the payment but right now i am very busy


In [5]:
list_of_intents = ["callback", "busy", "agree_to_pay","deny" ]


In [6]:
res = classifier(text, list_of_intents, multi_class=True)

In [7]:
res

{'labels': ['busy', 'agree_to_pay', 'callback', 'deny'],
 'scores': [0.9989303350448608,
  0.883718729019165,
  0.5817610621452332,
  0.09393595904111862],
 'sequence': 'ill make the payment but right now i am very busy'}

DEBUG:custom_connector:custom_connector: asr_text: ill make the payment but right now i am very busy
DEBUG:tracker_store_connection:Connecting to the Redis Tracker Store on : 11.0.1.32:6060
DEBUG:tracker_store_connection:Successfully Connected to the Redis Tracker Store !!!
2020-08-20 09:45:56 INFO     zero_shot  - {'sequence': 'ill make the payment but right now i am very busy', 'labels': ['busy', 'agree_to_pay', 'callback', 'deny', 'affirm', 'wrong_number'], 'scores': [0.9975331425666809, 0.6670709252357483, 0.4952879250049591, 0.4875871539115906, 0.44634366035461426, 0.07894844561815262]}
2020-08-20 09:45:56 INFO     zero_shot  - ****************Zero shot with threshold >  0.7 is: {'busy': 0.9975331425666809}
2020-08-20 09:45:56 INFO     zero_shot  - ****************Zero shot result is: {'sequence': 'ill make the payment but right now i am very busy', 'labels': ['busy', 'agree_to_pay', 'callback', 'deny', 'affirm', 'wrong_number'], 'scores': [0.9975331425666809, 0.6670709252357483, 0.4952879250049591, 0.4875871539115906, 0.44634366035461426, 0.07894844561815262]}
2020-08-20 09:45:56 INFO     actions  - <class 'dict'>
2020-08-20 09:45:56 INFO     actions  - MAX 2 = {'busy': 0.9975331425666809}


In [8]:
res['labels'][0]

'busy'

In [9]:
res['scores'][0]

0.9989303350448608

In [10]:
intent_dict = {res['labels'][0],res['scores'][0]  }

In [11]:
intent_dict

{0.9989303350448608, 'busy'}

In [12]:
intent_dict ={}

In [13]:
if res['scores'][0] >0.7  and res['scores'][1] >0.7:

  dic1 = {res['labels'][0]:res['scores'][0], res['labels'][1]:res['scores'][1]} 
  print("Current Dict is: ", dic1) 
else:
  print("*******")
  print(res['scores'][0])
  print(res['scores'][1])

  
# using the subscript notation 
# Dictionary_Name[New_Key_Name] = New_Key_Value 

Current Dict is:  {'busy': 0.9989303350448608, 'agree_to_pay': 0.883718729019165}


In [14]:
def zero_shot(sentence, list_of_intents ):
    res = classifier(sentence, list_of_intents, multi_class=True)
    print(res)
    if res['scores'][0] >0.7 and res['scores'][1] >0.7:
        dict1 = {res['labels'][0]:res['scores'][0], res['labels'][1]:res['scores'][1]} 
    else:
        dict1 = {res['labels'][0]:res['scores'][0]}
    print("Current Dict is: ", dict1) 
    print("**"*10)
    print("Zero shot with threshold >  0.7 is: "+str(dict1))
    print("**"*10)
    print("Zero shot result is: "+str(res))

    return(dict1)

In [15]:
zero_shot(text,list_of_intents)

{'sequence': 'ill make the payment but right now i am very busy', 'labels': ['busy', 'agree_to_pay', 'callback', 'deny'], 'scores': [0.9989303350448608, 0.883718729019165, 0.5817610621452332, 0.09393595904111862]}
Current Dict is:  {'busy': 0.9989303350448608, 'agree_to_pay': 0.883718729019165}
********************
Zero shot with threshold >  0.7 is: {'busy': 0.9989303350448608, 'agree_to_pay': 0.883718729019165}
********************
Zero shot result is: {'sequence': 'ill make the payment but right now i am very busy', 'labels': ['busy', 'agree_to_pay', 'callback', 'deny'], 'scores': [0.9989303350448608, 0.883718729019165, 0.5817610621452332, 0.09393595904111862]}


{'agree_to_pay': 0.883718729019165, 'busy': 0.9989303350448608}

##Examples

In [16]:
sequence = " yes i will pay not today but after 3 days"
candidate_labels = ["callback", "busy", "agree_to_pay","deny", "affirm"]

classifier(sequence, candidate_labels)

{'labels': ['affirm', 'agree_to_pay', 'callback', 'deny', 'busy'],
 'scores': [0.5457957983016968,
  0.2650972604751587,
  0.08514173328876495,
  0.0686410516500473,
  0.03532416373491287],
 'sequence': ' yes i will pay not today but after 3 days'}

*To* do multi-class classification, simply pass `multi_class=True`.                   In this case, the scores will be independent, but each will fall between 0 and 1.

In [17]:
sequence = " yes i will pay not today but after 3 days"
candidate_labels = ["callback", "busy", "agree_to_pay","deny", "affirm"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['agree_to_pay', 'affirm', 'callback', 'busy', 'deny'],
 'scores': [0.9874637126922607,
  0.9824212193489075,
  0.8924562931060791,
  0.7125455737113953,
  0.164438396692276],
 'sequence': ' yes i will pay not today but after 3 days'}

In [18]:
sequence = "I want agent help"
candidate_labels = ["callback", "busy", "seek_help"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['seek_help', 'busy', 'callback'],
 'scores': [0.9612129926681519, 0.9590674042701721, 0.04560573399066925],
 'sequence': 'I want agent help'}

Here's an example of sentiment classification: 

So how does this method work?

The underlying model is trained on the task of Natural Language Inference (NLI), which takes in two sequences and determines whether they contradict each other, entail each other, or neither.

This can be adapted to the task of zero-shot classification by treating the sequence which we want to classify as one NLI sequence (called the premise) and turning a candidate label into the other (the hypothesis). If the model predicts that the constructed premise _entails_ the hypothesis, then we can take that as a prediction that the label applies to the text. Check out [this blog post](https://joeddav.github.io/blog/2020/05/29/ZSL.html) for a more detailed explanation.

By default, the pipeline turns labels into hypotheses with the template `This example is {class_name}.`. This works well in many settings, but you can also customize this for your specific setting.

A premise is the foundation of a logical argument, a fact that both parties agree to. Sound arguments use only their premises to reach a conclusion.
Premise is a statement or a scenario and


A hypothesis is a suggested explanation for something.
Hypothesis is a statement that is tested in a scientific experiment.


##Sentiment Analysis

So how does this method work?

The underlying model is trained on the task of Natural Language Inference (NLI), which takes in two sequences and determines whether they contradict each other, entail each other, or neither.

This can be adapted to the task of zero-shot classification by treating the sequence which we want to classify as one NLI sequence (called the premise) and turning a candidate label into the other (the hypothesis). If the model predicts that the constructed premise _entails_ the hypothesis, then we can take that as a prediction that the label applies to the text. Check out [this blog post](https://joeddav.github.io/blog/2020/05/29/ZSL.html) for a more detailed explanation.

By default, the pipeline turns labels into hypotheses with the template `This example is {class_name}.`. This works well in many settings, but you can also customize this for your specific setting. Let's add another review to our above sentiment classification example that's a bit more challenging:

In [19]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]

classifier(sequences, candidate_labels)

[{'labels': ['negative', 'positive'],
  'scores': [0.9916267991065979, 0.008373182266950607],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['negative', 'positive'],
  'scores': [0.8148515820503235, 0.1851484179496765],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

The second example is a bit harder. Let's see if we can improve the results by using a hypothesis template which is more specific to the setting of review sentiment analysis. Instead of the default, `This example is {}.`, we'll use, `The sentiment of this review is {}.` (where `{}` is replaced with the candidate class name)

In [20]:
sequences = [
    "I hated this movie. The acting sucked.",
    "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
]
candidate_labels = ["positive", "negative"]
hypothesis_template = "The sentiment of this review is {}."

classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

[{'labels': ['negative', 'positive'],
  'scores': [0.9890093207359314, 0.010990672744810581],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['positive', 'negative'],
  'scores': [0.9581228494644165, 0.0418771356344223],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

By providing a more precise hypothesis template, we are able to see a more accurate classification of the second review.

> Note that sentiment classification is used here just as an illustrative example. The [Hugging Face Model Hub](https://huggingface.co/models?filter=text-classification) has a number of models trained specifically on sentiment tasks which can be used instead.

##sentence 2

In [21]:
sequence = "Yes, I will pay online tomorrow"
candidate_labels = ["callback", "payment_mode", "agree_to_pay"]

classifier(sequence, candidate_labels)

{'labels': ['agree_to_pay', 'callback', 'payment_mode'],
 'scores': [0.6090993285179138, 0.2011737823486328, 0.1897268444299698],
 'sequence': 'Yes, I will pay online tomorrow'}

To do multi-class classification, simply pass `multi_class=True`. In this case, the scores will be independent, but each will fall between 0 and 1.

In [22]:
sequence = "Yes, I will pay online tomorrow"
candidate_labels = ["callback", "payment_mode", "agree_to_pay"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['agree_to_pay', 'payment_mode', 'callback'],
 'scores': [0.9814551472663879, 0.8815153241157532, 0.5569513440132141],
 'sequence': 'Yes, I will pay online tomorrow'}

##sentence 3

In [23]:
sequence = "Yes, I will pay online tomorrow but i am busy right now"
candidate_labels = ["callback", "busy", "agree_to_pay"]

classifier(sequence, candidate_labels)

{'labels': ['busy', 'agree_to_pay', 'callback'],
 'scores': [0.9068845510482788, 0.06753562390804291, 0.02557981386780739],
 'sequence': 'Yes, I will pay online tomorrow but i am busy right now'}

To do multi-class classification, simply pass `multi_class=True`. In this case, the scores will be independent, but each will fall between 0 and 1.

In [24]:
sequence = "Yes, I will pay online tomorrow but i am busy right now"
candidate_labels = ["callback", "busy", "agree_to_pay"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['busy', 'agree_to_pay', 'callback'],
 'scores': [0.9991379976272583, 0.9591472148895264, 0.5140132904052734],
 'sequence': 'Yes, I will pay online tomorrow but i am busy right now'}

##sentence 4

In [25]:
sequence = "I need a cheque book containing 20 leaves but right now i am busy"
candidate_labels = ["need_checkbook", "busy", "callback"]

classifier(sequence, candidate_labels)

{'labels': ['busy', 'need_checkbook', 'callback'],
 'scores': [0.8570942282676697, 0.13683098554611206, 0.006074815057218075],
 'sequence': 'I need a cheque book containing 20 leaves but right now i am busy'}

To do multi-class classification, simply pass `multi_class=True`. In this case, the scores will be independent, but each will fall between 0 and 1.

In [26]:
sequence = "I need a cheque book containing 20 leaves but right now i am busy"
candidate_labels = ["need_checkbook", "busy", "callback"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['busy', 'need_checkbook', 'callback'],
 'scores': [0.9992415308952332, 0.973717212677002, 0.01672358624637127],
 'sequence': 'I need a cheque book containing 20 leaves but right now i am busy'}

Here's an example of sentiment classification: 

##sentence 5

In [27]:
sequence = " I will pay but not today Please remind me tomorrow"
candidate_labels = ["agree_to_pay", "busy", "callback"]

classifier(sequence, candidate_labels)

{'labels': ['agree_to_pay', 'callback', 'busy'],
 'scores': [0.4571768343448639, 0.4289582669734955, 0.11386487632989883],
 'sequence': ' I will pay but not today Please remind me tomorrow'}

To do multi-class classification, simply pass `multi_class=True`. In this case, the scores will be independent, but each will fall between 0 and 1.

In [28]:
sequence = "I will pay but not today Please remind me tomorrow"
candidate_labels = ["agree_to_pay", "busy", "callback"]

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['agree_to_pay', 'callback', 'busy'],
 'scores': [0.9663274884223938, 0.9446771740913391, 0.8788650035858154],
 'sequence': 'I will pay but not today Please remind me tomorrow'}

## experiments

In [29]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

# Sentiment analysis pipeline
pipeline('sentiment-analysis')

# Question answering pipeline, specifying the checkpoint identifier
pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='bert-base-cased')

# Named entity recognition pipeline, passing in a specific model and tokenizer
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
pipeline('ner', model=model, tokenizer=tokenizer)

In [30]:
sequence = "no i cannot pay now i will pay after two days"
candidate_labels = ["deny", "callback", "affirm", 'agree_to_pay', 'busy']

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['deny', 'busy', 'callback', 'affirm', 'agree_to_pay'],
 'scores': [0.9953843951225281,
  0.9890685081481934,
  0.6064962148666382,
  0.06882078945636749,
  0.05890253558754921],
 'sequence': 'no i cannot pay now i will pay after two days'}

In [31]:
sequence = "no i cannot pay"
candidate_labels = ["deny", "callback", "affirm", 'agree_to_pay', 'busy']

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['deny', 'busy', 'callback', 'affirm', 'agree_to_pay'],
 'scores': [0.997624933719635,
  0.9669301509857178,
  0.21569079160690308,
  0.08682606369256973,
  0.005708230659365654],
 'sequence': 'no i cannot pay'}

In [32]:
sequence = "i will pay after two days"
candidate_labels = ["deny", "callback", "affirm", 'agree_to_pay', 'busy']

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['agree_to_pay', 'callback', 'affirm', 'deny', 'busy'],
 'scores': [0.9695858359336853,
  0.7340303659439087,
  0.5304573178291321,
  0.48545876145362854,
  0.4521147310733795],
 'sequence': 'i will pay after two days'}

In [33]:
sequence = "yes i will pay now"
candidate_labels = ["deny", "callback", "affirm", 'agree_to_pay', 'busy']

classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['agree_to_pay', 'affirm', 'callback', 'busy', 'deny'],
 'scores': [0.994513750076294,
  0.9927086234092712,
  0.902932345867157,
  0.7542465329170227,
  0.004379716701805592],
 'sequence': 'yes i will pay now'}

0

In [ ]:
sequence = "ill make the payment but right now i am very busy"
candidate_labels = ["deny", "callback", "affirm", 'agree_to_pay', 'busy']
classifier(sequence, candidate_labels, multi_class=True)